In [ ]:
## 한글 맞춤법 확인하는 라이브러리
# !pip install git+https://github.com/jungin500/py-hanspell

In [ ]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

# 데이터 로드
| 파일 이름 | 데이터 설명 |
| ----------| ------------|
| 0. 구_과_하는일_통합.csv | 과별 하는 일을 담아놓은 데이터셋 |
| 0. 새올_통합.csv | 구별 민원 데이터셋 |
| 삭제된 과 대체.xlsx | 과 이름이 바뀌어서 민원 데이터셋에는 존재하는 과지만 과별 하는 일 데이터셋에는 존재하지 않는 과를 임의로 맵핑해놓은 데이터셋|

In [ ]:
gwa_df = pd.read_csv('..\\data\\0. 구_과_하는일_통합.csv')
gwa_df.shape, gwa_df.head(2)

((934, 4),
      구    국      과                                                하는일
 0  광진구  행정국    총무과  광진구 의전지침 수립 및 현안 관리 등 신년인사회, 구민의 날, 정례조례 등 구 주...
 1  광진구  행정국  자치행정과  동 행정 종합계획 동 직원 근무평정 조정 고향사랑기부제 동 지역책임제 사업 동 행정...)

In [ ]:
minwon_df = pd.read_csv('..\\data\\0. 새올_통합.csv')
minwon_df.shape, minwon_df.head(2)

((124698, 11),
      req_id           req_title req_p             req_date  \
 0  445530.0       양재천 너구리(?) 출현   홍영규  2023-07-31 08:42:53   
 1  444933.0  2415번 버스기사님 칭찬합니다.   백○○  2023-07-28 13:11:17   
 
                                          req_content    resp_dept  \
 0  어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...  도시환경국 공원녹지과   
 1  평일 오전 포스코방면 2415번 버스를 운행하시는 박형춘 기사님을 칭찬합니다.\n기...        서울특별시   
 
              resp_date resp_writer  \
 0  2023-08-01 18:01:12         이지영   
 1                  NaN         NaN   
 
                                         resp_content  page_num gu_name  
 0  안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...         1     강남구  
 1                                                NaN         1     강남구  )

In [ ]:
replace_gwa_df = pd.read_excel('..\\data\\raw data\\삭제된 과 대체.xlsx')
replace_gwa_df.shape, replace_gwa_df.head(2)

((77, 4),
   gu_name resp_dept  count changed_dept
 0     강남구     공동주택과     37          주택과
 1     강남구     교통정책과    281        교통행정과)

In [ ]:
minwon_df

,req_id,req_title,req_p,req_date,req_content,resp_dept,resp_date,resp_writer,resp_content,page_num,gu_name
0,445530.0,양재천 너구리(?) 출현,홍영규,2023-07-31 08:42:53,어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...,도시환경국 공원녹지과,2023-08-01 18:01:12,이지영,"안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...",1,강남구
1,444933.0,2415번 버스기사님 칭찬합니다.,백○○,2023-07-28 13:11:17,평일 오전 포스코방면 2415번 버스를 운행하시는 박형춘 기사님을 칭찬합니다.\n기...,서울특별시,NaN,NaN,NaN,1,강남구
2,444484.0,통신판매업 신고 - 여러 사이트의 경우,김○○,2023-07-27 10:56:28,안녕하세요\n통신판매업 신고가 이미 되어있는 상태입니다.(a쇼핑몰 주소)\n같은 통...,기획경제국 지역경제과,2023-08-01 18:07:52,김나연,"1. 우리구정에 깊은 관심과 애정을 보여 주신데 대하여 감사드리며, 접수번호(202...",1,강남구
3,444462.0,그랜드 인터컨티넨탈 서울 파르나스 앞 인도 건널목 교통정리 불법 여부,김○○,2023-07-27 09:56:28,그랜드 인터컨티넨탈 서울 파르나스 앞 인도 건널목에서 호텔측 고용인으로 보이시는 분...,경찰청,NaN,NaN,NaN,1,강남구
4,444205.0,헌릉로(서울 강남구 세곡동 79-6)도로 안전지대 조정 요청,유○○,2023-07-26 11:41:23,"도로 주소: 서울 강남구 세곡동 79-6 (헌릉로, 위례터널에서 세곡동사거리 방향)...",서울특별시,NaN,NaN,NaN,1,강남구
...,...,...,...,...,...,...,...,...,...,...,...
124693,1124.0,중랑구 빈병회수 에대한 법 22조 2항을 위반한 업체들을 처분해 주십시요.,강상길,2007-12-31 13:09:20,공병회수 에 대해 환경부에 밑에 민원상담을 했습니다.\n\n저히집은 애주가 집안이라...,주민생활지원국 청소행정과,2008-01-03 09:15:38,김용식,- 고객님 안녕하십니까?\n\n- 먼저 고객님의 빈병 보증금 회수 관계로 불편을 드...,145,중랑구
124694,1112.0,길고양이 TNR관련,백송이,2007-11-29 10:23:33,안녕하세요.\n내년부터 서울시에서 길고양이 TNR을 실시한다는 얘기를 들었습니다.\...,재정경제국 지역경제과,2007-12-04 15:08:27,김종수,안녕하세요?\n고양이 문제에 대하여 많은 애정과 관심을 가지고 좋은 의견을 제시해 ...,145,중랑구
124695,1111.0,중랑구청 식당,조병현,2007-11-28 13:25:08,중랑구청에서 점심식사를 했습니다.\n식권 3000원.\n반찬 김치.새우넣은마늘쫑 볶...,행정국 총무과,2007-12-06 14:40:48,김진하,○ 구내 식당운영에 관심을 가져주신 고객님께 감사를 드립니다.\n\n○ 우리 구청 ...,145,중랑구
124696,1107.0,제목: 쇼핑몰 창업에 대한 문의,박윤정,2007-11-20 02:13:01,1.과일즙을 판매하는 쇼핑몰을 창업하려고 하는데요.\n오프라인에 가게가 없이 집에서...,재정경제국 지역경제과,2007-11-21 11:02:17,임영은,중랑구청 지역경제과 통신판매담당 임영은 입니다.\n\n문의하신 내용중 1번 질문과 ...,145,중랑구


# 과 하는일 전처리
- 한국어 추출

In [ ]:
# 한국어만 추출
def extract_korean_text(text):
    if isinstance(text, str):
        korean_pattern = re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣]+')
        korean_words = korean_pattern.findall(text)
        return ' '.join(korean_words)
    else:
        return ''

In [ ]:
# 한국어만 추출
gwa_df['하는일'] = gwa_df['하는일'].apply(extract_korean_text)

In [ ]:
gwa_df.to_csv('../data/0. 구_과_하는일_통합.csv', index=False, encoding='utf-8-sig')

# 민원 전처리
- 결측치 처리
    - 민원글이 없는 경우 제거
    - 이송이첩 제외(이송이첩인 데이터는 null값으로 넣은 컬럼이 있으므로 해당 컬럼 사용할 것)
- 이송이첩 제외
  - 결측치로 제외하지 못한 경우가 있음
  - ~과, %보건%, %안전센터% 만 남기기
- ~~한국어 추출~~ -> **대부분의 특수기호는 BERT 모델에서 제거됨**
- ~~맞춤법 확인~~ -> **맞춤법 없이 넣는 것이 더 높은 성능 보임**
- 민원 제목을 글에 넣기
- 답변 부서에서 국과 과 구분
- 변경된 과 이름 이름 대체

## 결측치 처리
- 우리 태스크에서 중요한 두 변수는 민원자가 작성한 민원글(req_content)과 민원에 대한 처리를 한 부서(resp_dept)이다. 이 두 컬럼이 null이 아니게끔 바꾸기
  - 민원글(req_content)이 null이면 안됨 -> req_id, req_title, req_p, req_date, req_content 해결
  - 답변 부서(resp_dept)가 null인 경우는 이송이첩이기 때문에 제거해도 됨(이송이첩 기준은 resp_date가 null인지로 판단)-> resp_date, resp_writer, resp_content 해결

In [ ]:
def missing_values(df):
    print('#################### 결측치 처리 시작 ####################')
    print('### 결측치 처리 전 ### \n',df.isnull().sum())
    cols = ['req_content', 'resp_date'] # 결측치를 확인할 기준이 되는 컬럼

    for col in cols:
        drop_idx = df[df[col].isnull()].index
        print('\n### {} 컬럼 결측치 처리 ### \n삭제할 row 개수 : '.format(col), len(drop_idx))
        df.drop(drop_idx, axis=0, inplace=True)
        df.reset_index(drop=True, inplace=True)

    df['resp_content'].fillna('', inplace=True) # 이송이첩이 아닌데 resp_content가 결측치인 경우가 있음 -> 공백으로 넣기
    df.reset_index(drop=True, inplace=True)
    print('\n### 결측치 처리 후 ### \n', df.isnull().sum())
    print('\n### 결측치 처리 후 ### \n', df.shape)
    return df

## 이송이첩 처리
- ~과, %보건%, %안전센터% 만 남기기 -> 보건, 안전센터는 과 df에 있기 때문에 걸러냄

In [ ]:
# ~과, %보건%, %안전센터%이면 그대로, 아니면 공백
def filtering_gwa(txt):
    if re.search('과|보건|안전센터', txt):
        return txt
    else :
        return ''

# 위에서 만든 함수 사용 -> 공백이면 제거하기
def sended_organ(df):
    drop_idx = df[df['resp_dept'].apply(filtering_gwa) == '']['resp_dept'].index
    print('나머지 이송이첩 \n삭제할 row 개수 : ', len(drop_idx))
    df.drop(drop_idx, axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print('\n### 이송이첩 처리 후 ### \n', df.shape)
    return df

## hanspell

- 맞춤법 교정 라이브러리인 [hanspell](https://github.com/ssut/py-hanspell) 사용
- **최초에는 맞춤법 교정을 하려 했으나, 안하는 것이 더 큰 성능을 보이므로 진행 x**
- 500자 이상 넘어가는 텍스트는 판별할 수 없기 때문에 500자씩 끊어서 진행

In [ ]:
# import hanspell
# from hanspell import spell_checker

In [ ]:
# 민원 글에 대하여 맞춤법과 띄어쓰기 체크
# from hanspell import spell_checker
def check_spell(txt):
    from hanspell import spell_checker # 필요한 라이브러리 import
    try:
        checked_txt = spell_checker.check(txt)  # 맞춤법, 띄어쓰기 체크
        result = checked_txt.as_dict()['checked'] # 올바르게 교정한 결과
    except:
        result = txt

    return result

In [ ]:
# 띄어쓰기 기준으로 나눈 후 500글자씩 끊기 -> hanspell이 500글자까지 넣을 수 있으므로
def split_500_char(txt):
    i = 0 # 인덱스 초기화
    result_list = list() # 500글자씩 나눠진 텍스트를 담을 리스트

    split_list = txt.split(' ') # 입력받은 텍스트를 띄어쓰기 단위로 나누기

    while i <len(split_list):
        temp_text = '' # 반복마다 초기화
        while i < len(split_list):
            if len(temp_text + ' ' + split_list[i]) < 500: # 한 덩어리의 길이가 500 이하여야 함
                temp_text = temp_text + ' ' + split_list[i] # 다음 단어를 더해서 업데이트
                i += 1
            else:
                break
        result_list.append(temp_text.replace(' ','', 1)) # 맨 앞에 들어가있는 공백 지우기

    return result_list

In [ ]:
# 위 두 함수를 결합해 모든 민원글에 대하여 맞춤법 검사 -> 500글자 이하인 데이터와 이상인 데이터 따로 처리해서 concat
def check_spell_final(df):
    print('\n#################### 맞춤법 처리 시작 ####################')
    in_500_df = df[df['req_content'].str.len() < 501] # 500자를 넘지 않는 데이터
    out_500_df = df.drop(in_500_df.index, axis=0)            # 500자를 넘는 데이터

    in_500_df.reset_index(drop=True, inplace=True)
    out_500_df.reset_index(drop=True, inplace=True)

    # 500자 이하인 데이터는 그냥 apply
    print('\n### 500자 이하 민원글 맞춤법 검사 시작 ###')
    print('500자 이하 데이터프레임 shape : ', in_500_df.shape)
    in_500_df['typo_req_content'] = in_500_df['req_content'].apply(check_spell)

    # 500자 초과인 데이터는 각각 500자씩 끊어서 맞춤법 검사 해야함
    print('\n### 500자 초과하는 민원글 맞춤법 검사 시작 ###')
    print('500자 초과 데이터프레임 shape : ', out_500_df.shape)
    content_list = out_500_df['req_content'].values # 민원글만 array로 가져오기
    result_list  = np.array([]) # 결과를 담을 array

    # 이상하게 가끔씩 JSONDecodeError에러가 뜨는 데이터가 있는데, 이런 데이터는 그냥 원본 데이터로 추가
    for i in tqdm(range(len(content_list))):
        text = content_list[i] # 하니싹 민원글 가져오기
        typo_text = ' '.join(pd.Series(split_500_char(text)).apply(check_spell)) # 500글자씩 나눠서 seires로 나눈 후 맞춤법 검사
        result_list = np.append(result_list, typo_text)

    # 데이터프레임에 추가
    out_500_df['typo_req_content'] = result_list

    # 두 데이터프레임을 하나로 합치기
    df = pd.concat([in_500_df, out_500_df], axis=0)
    df.reset_index(drop=True, inplace=True)
    print('\n### 맞춤법 처리 후 ### \n', df.shape)

    return df

In [ ]:
# 100개의 데이터로 한 번 돌려보기
# temp_df = minwon_df.copy()
# temp_df = temp_df.loc[:100]

# result_df = check_spell_final(temp_df)
# result_df

## 민원글에 제목 추가하기

In [ ]:
def concat_req_title_content(df):
    print('\n#################### 제목 + 글 병합 시작 ####################')
    df['req_title'] = df['req_title'].apply(extract_korean_text)
    df['req_content'] = df['req_title'] + ' ' + df['req_content']
    return df

## 답변 부서에서 국과 과 구분
- 띄어쓰기로 구분이 되어있으므로 띄어쓰기를 기준으로 할 것

In [ ]:
def split_guk_gwa(df):
    print('\n#################### 국/과 분리 시작 ####################')
    # 띄어쓰기를 기준으로 나누고 마지막 요소가 과 이름임
    df['resp_dept'] = df['resp_dept'].str.split(' ').str[-1]

    return df

## 변경된 과 이름 이름 대체
- '0. 삭제된 과 대체.xlsx'를 기준으로 바꾸기
- 딕셔너리 형태로 사용하긴 하지만, 별도의 딕셔너리를 만들진 않을 것임

In [ ]:
def replace_gwa_name(df, dict_df): # 바꿀 데이터프레임, 기준이 되는 데이터프레임
    print('\n#################### 과 이름 대체 시작 ####################')
    # {(구 이름, 이전 과 이름) : 변경된 과이름} 형태로 만들어주기(딕셔너리 형태로) / 딕셔너리를 사용할 것은 아님
    # temp_df의 인덱스가 (구이름, 이전 과 이름)인 멀티인덱스로 되어있음
    temp_df = dict_df.groupby(['gu_name', 'resp_dept', 'changed_dept']).size().to_frame().reset_index('changed_dept')

    # 민원 데이터프레임에 key 컬럼 추가 -> 임시로 사용할 컬럼임
    df['key'] = "('" + df['gu_name'] + "', '" + df['resp_dept'] + "')"

    for i in range(len(temp_df)):
        key = temp_df.index[i]
        val = temp_df['changed_dept'].iloc[i]

        # key는 string으로 데이터형 변환
        change_idx = df[df['key'] == str(key)].index # 바꿔야할 인덱스 가져오기
        df.loc[change_idx, 'resp_dept'] = val # 위에서 가져온 인덱스에 맞춰서 과 이름 변경

    # key 컬럼 삭제
    df.drop('key', axis=1, inplace=True)
    print('### 과 이름 변경 건 처리 완료 ### ')
    return df

## 민원 글의 과가 없는 경우 제외하기
 - merge를 하면  동작구 치수과에서 총 144개의 데이터가 추가됨
- 제거해주기(기준 열은 ['req_title', 'req_content','req_date', 'resp_date', 'resp_dept'])


In [ ]:
def merge_minwon_gwa(df, gwa_df): # 민원, 과 순으로 input
    print('\n#################### 민원글, 과 merge 시작 ####################')

    # merge를 위해 과 하는일 데이터프레임의 컬럼 이름을 바꾸기
    cols = ['gu_name', '국', 'resp_dept', '하는일']
    gwa_df.columns = cols

    # 구, 과를 key로 left join
    merge_df = pd.merge(df, gwa_df, on=['gu_name', 'resp_dept'], how='left')

    # join하면 중복되는 데이터가 일부 있기 때문에 지워주기
    merge_df.drop_duplicates(['req_title', 'req_content','req_date', 'resp_date', 'resp_dept'], inplace=True)

    # 과 리스트에 해당하는 과가 없는 민원글은 제거하기
    drop_idx = merge_df[merge_df['하는일'].isnull()].index
    drop_df = merge_df.loc[drop_idx] # 삭제 대상인 데이터프레임 따로 저장
    result_df = merge_df.drop(drop_idx, axis=0) # 결과 테이블로 반환할 데이터프레임

    drop_df.reset_index(drop=True, inplace=True)
    result_df.reset_index(drop=True, inplace=True)

    print('\n### 과, 민원의 과 병합 후 ###\n', result_df.shape)

    return result_df, drop_df

# 한 번에 처리하기
- 위에서 만든 모듈 한 번에 실행

In [ ]:
minwon_df = missing_values(minwon_df)                                          # 결측치 처리
minwon_df = sended_organ(minwon_df)                                            # 이송이첩 처리
minwon_df = split_guk_gwa(minwon_df)                                           # 답변 부서 국과 구 분리
minwon_df = replace_gwa_name(minwon_df, replace_gwa_df)                        # 변경된 과 이름 대체
minwon_df, drop_df = merge_minwon_gwa(minwon_df, gwa_df)                       # 과 하는일, 민원의 과 맵핑
minwon_df = concat_req_title_content(minwon_df)                                # 제목이랑 내용이랑 합치기

#################### 결측치 처리 시작 ####################
### 결측치 처리 전 ### 
 req_id           2207
req_title        2207
req_p            2207
req_date         2207
req_content      2208
resp_dept           0
resp_date       13677
resp_writer     13677
resp_content    13682
page_num            0
gu_name             0
dtype: int64

### req_content 컬럼 결측치 처리 ### 
삭제할 row 개수 :  2208

### resp_date 컬럼 결측치 처리 ### 
삭제할 row 개수 :  13492

### 결측치 처리 후 ### 
 req_id          0
req_title       0
req_p           0
req_date        0
req_content     0
resp_dept       0
resp_date       0
resp_writer     0
resp_content    0
page_num        0
gu_name         0
dtype: int64

### 결측치 처리 후 ### 
 (108998, 11)
나머지 이송이첩 
삭제할 row 개수 :  5089

### 이송이첩 처리 후 ### 
 (103909, 11)

#################### 국/과 분리 시작 ####################

#################### 과 이름 대체 시작 ####################
### 과 이름 변경 건 처리 완료 ### 

#################### 민원글, 과 merge 시작 ####################

### 과, 민원의 과 병합 후 ###
 (88039, 13)


(datetime.date(2023, 8, 22), datetime.time(16, 2, 33, 321349))

In [ ]:
minwon_df.shape, drop_df.shape

((88039, 13), (15870, 13))

In [ ]:
import os

os.listdir('..\\data')

['0. 구_과_하는일_통합.csv',
 '0. 새올_통합.csv',
 'droped data.csv',
 'raw data',
 '과 하는일 tidy data']

In [ ]:
minwon_df.to_csv('..\\data\\0. 새올_통합.csv', index=False, encoding='utf-8-sig')
drop_df.to_csv('..\\data\\droped data.csv', index=False, encoding='utf-8-sig')

# 학습 데이터 만들기

In [ ]:
minwon_df

,req_id,req_title,req_p,req_date,req_content,resp_dept,resp_date,resp_writer,resp_content,page_num,gu_name,국,하는일
0,445530.0,양재천 너구리(?) 출현,홍영규,2023-07-31 08:42:53,어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...,공원녹지과,2023-08-01 18:01:12,이지영,"안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...",1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...
1,444484.0,통신판매업 신고 - 여러 사이트의 경우,김○○,2023-07-27 10:56:28,안녕하세요\n통신판매업 신고가 이미 되어있는 상태입니다.(a쇼핑몰 주소)\n같은 통...,지역경제과,2023-08-01 18:07:52,김나연,"1. 우리구정에 깊은 관심과 애정을 보여 주신데 대하여 감사드리며, 접수번호(202...",1,강남구,기획경제국,경제동향 분석 세곡천 힐링텃밭 운영 및 관리 스마트팜 조성 및 운영 과 주요사업 및...
2,443723.0,도로 노면 홈으로 인한 물튀김 피해,O○○,2023-07-24 21:19:42,지난 화요일(7/18) 오전 8:50~8:55분경 압구정로 30길 I.S.A빌딩과 ...,도로관리과,2023-08-01 16:46:45,최종현,1. 안녕하십니까. 먼저 우리 구 도로에서 물튀김 피해를 입으신 것에 대해 안타깝게...,1,강남구,안전교통국,구의회 업무 총무 인사 보안 직원복무 교육 복리후생 급여 및 제수당 청렴 물품관리 ...
3,443578.0,실내 공기질 측정기 대여 서비스,김○○,2023-07-24 14:38:41,연일 노고에 감사드립니다\n\n올해 개포동에 신축아파트들이 많이 지어집니다\n실내공...,환경과,2023-07-28 08:52:35,노서우,안녕하십니까? 평소 구정 발전에 적극 협조하여 주신 귀하께 깊이 감사드립니다. 귀하...,1,강남구,도시환경국,그린 스마트 쉼터 신설 및 관리 압구정로 도산대로 학동로 기후변화 취약계층 지원사업...
4,442559.0,이동약자의 이동공간 확보를 도와주세요,정○○,2023-07-21 10:37:24,붙임 사진을 제 아버님이 거주하시는 개포우성2차 아파트 출입구 사진입니다.\n보시면...,주택과,2023-07-31 17:38:48,김혜숙,1.안녕하십니까? 귀하께서 신청하신 민원에 대해 다음과 같이 안내드립니다.\n2.귀...,1,강남구,도시환경국,주요업무계획 수립 국예산 및 결산 국 장 회의자료 국 인사 근평 국 소관 구의회 업...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88034,1136.0,불법주차가 너무 심해서 차가 너무 막혀요,박종수,2008-01-13 00:02:53,불법주차를 가장 잘 단속해야 하는 지역이 중랑구청 앞이란 당연한 생각이 듭니다.\n...,교통행정과,2008-01-18 13:17:45,김충만,○ 항상 구정발전에 관심을 가져주신 고객님의 가정에 행운이 가득하시길 기원합니다.\...,145,중랑구,안전건설교통국,교통유발부담금 화물인허가 택시 인허가 어린이보호구역 개선사업 교통체계개선사업 교통체...
88035,1135.0,어이없어서 이렇게 글을 올립니다.,문수철,2008-01-11 17:10:41,안녕하세요\n면목5동 에서 슈퍼를 운영하고 있는 사람입니다.\n다름이 아니라 슈퍼를...,지역경제과,2008-01-18 14:18:25,이종희,○ 구 행정에 많은 관심과 협조를 해주시는 문수철님께 감사를 드립니다.\n○ 귀하의...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...
88036,1124.0,중랑구 빈병회수 에대한 법 22조 2항을 위반한 업체들을 처분해 주십시요.,강상길,2007-12-31 13:09:20,공병회수 에 대해 환경부에 밑에 민원상담을 했습니다.\n\n저히집은 애주가 집안이라...,청소행정과,2008-01-03 09:15:38,김용식,- 고객님 안녕하십니까?\n\n- 먼저 고객님의 빈병 보증금 회수 관계로 불편을 드...,145,중랑구,생활복지국,종량제 봉투 제작 및 공급 세외수입 중간집하장 이전 폐기물처리시설 건립기금관리 재활...
88037,1112.0,길고양이 TNR관련,백송이,2007-11-29 10:23:33,안녕하세요.\n내년부터 서울시에서 길고양이 TNR을 실시한다는 얘기를 들었습니다.\...,지역경제과,2007-12-04 15:08:27,김종수,안녕하세요?\n고양이 문제에 대하여 많은 애정과 관심을 가지고 좋은 의견을 제시해 ...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...


In [ ]:
minwon_df['req_title_content'] = minwon_df['req_title'] + ' ' + minwon_df['req_content']
minwon_df

,req_id,req_title,req_p,req_date,req_content,resp_dept,resp_date,resp_writer,resp_content,page_num,gu_name,국,하는일,req_title_content
0,445530.0,양재천 너구리(?) 출현,홍영규,2023-07-31 08:42:53,어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...,공원녹지과,2023-08-01 18:01:12,이지영,"안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...",1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...
1,444484.0,통신판매업 신고 - 여러 사이트의 경우,김○○,2023-07-27 10:56:28,안녕하세요\n통신판매업 신고가 이미 되어있는 상태입니다.(a쇼핑몰 주소)\n같은 통...,지역경제과,2023-08-01 18:07:52,김나연,"1. 우리구정에 깊은 관심과 애정을 보여 주신데 대하여 감사드리며, 접수번호(202...",1,강남구,기획경제국,경제동향 분석 세곡천 힐링텃밭 운영 및 관리 스마트팜 조성 및 운영 과 주요사업 및...,통신판매업 신고 - 여러 사이트의 경우 안녕하세요\n통신판매업 신고가 이미 되어있는...
2,443723.0,도로 노면 홈으로 인한 물튀김 피해,O○○,2023-07-24 21:19:42,지난 화요일(7/18) 오전 8:50~8:55분경 압구정로 30길 I.S.A빌딩과 ...,도로관리과,2023-08-01 16:46:45,최종현,1. 안녕하십니까. 먼저 우리 구 도로에서 물튀김 피해를 입으신 것에 대해 안타깝게...,1,강남구,안전교통국,구의회 업무 총무 인사 보안 직원복무 교육 복리후생 급여 및 제수당 청렴 물품관리 ...,도로 노면 홈으로 인한 물튀김 피해 지난 화요일(7/18) 오전 8:50~8:55분...
3,443578.0,실내 공기질 측정기 대여 서비스,김○○,2023-07-24 14:38:41,연일 노고에 감사드립니다\n\n올해 개포동에 신축아파트들이 많이 지어집니다\n실내공...,환경과,2023-07-28 08:52:35,노서우,안녕하십니까? 평소 구정 발전에 적극 협조하여 주신 귀하께 깊이 감사드립니다. 귀하...,1,강남구,도시환경국,그린 스마트 쉼터 신설 및 관리 압구정로 도산대로 학동로 기후변화 취약계층 지원사업...,실내 공기질 측정기 대여 서비스 연일 노고에 감사드립니다\n\n올해 개포동에 신축아...
4,442559.0,이동약자의 이동공간 확보를 도와주세요,정○○,2023-07-21 10:37:24,붙임 사진을 제 아버님이 거주하시는 개포우성2차 아파트 출입구 사진입니다.\n보시면...,주택과,2023-07-31 17:38:48,김혜숙,1.안녕하십니까? 귀하께서 신청하신 민원에 대해 다음과 같이 안내드립니다.\n2.귀...,1,강남구,도시환경국,주요업무계획 수립 국예산 및 결산 국 장 회의자료 국 인사 근평 국 소관 구의회 업...,이동약자의 이동공간 확보를 도와주세요 붙임 사진을 제 아버님이 거주하시는 개포우성2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88034,1136.0,불법주차가 너무 심해서 차가 너무 막혀요,박종수,2008-01-13 00:02:53,불법주차를 가장 잘 단속해야 하는 지역이 중랑구청 앞이란 당연한 생각이 듭니다.\n...,교통행정과,2008-01-18 13:17:45,김충만,○ 항상 구정발전에 관심을 가져주신 고객님의 가정에 행운이 가득하시길 기원합니다.\...,145,중랑구,안전건설교통국,교통유발부담금 화물인허가 택시 인허가 어린이보호구역 개선사업 교통체계개선사업 교통체...,불법주차가 너무 심해서 차가 너무 막혀요 불법주차를 가장 잘 단속해야 하는 지역이 ...
88035,1135.0,어이없어서 이렇게 글을 올립니다.,문수철,2008-01-11 17:10:41,안녕하세요\n면목5동 에서 슈퍼를 운영하고 있는 사람입니다.\n다름이 아니라 슈퍼를...,지역경제과,2008-01-18 14:18:25,이종희,○ 구 행정에 많은 관심과 협조를 해주시는 문수철님께 감사를 드립니다.\n○ 귀하의...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...,어이없어서 이렇게 글을 올립니다. 안녕하세요\n면목5동 에서 슈퍼를 운영하고 있는 ...
88036,1124.0,중랑구 빈병회수 에대한 법 22조 2항을 위반한 업체들을 처분해 주십시요.,강상길,2007-12-31 13:09:20,공병회수 에 대해 환경부에 밑에 민원상담을 했습니다.\n\n저히집은 애주가 집안이라...,청소행정과,2008-01-03 09:15:38,김용식,- 고객님 안녕하십니까?\n\n- 먼저 고객님의 빈병 보증금 회수 관계로 불편을 드...,145,중랑구,생활복지국,종량제 봉투 제작 및 공급 세외수입 중간집하장 이전 폐기물처리시설 건립기금관리 재활...,중랑구 빈병회수 에대한 법 22조 2항을 위반한 업체들을 처분해 주십시요. 공병회수...
88037,1112.0,길고양이 TNR관련,백송이,2007-11-29 10:23:33,안녕하세요.\n내년부터 서울시에서 길고양이 TNR을 실시한다는 얘기를 들었습니다.\...,지역경제과,2007-12-04 15:08:27,김종수,안녕하세요?\n고양이 문제에 대하여 많은 애정과 관심을 가지고 좋은 의견을 제시해 ...,145,중랑구,기획재정국,골목형상점가 시설현대화 사업 공모사업 및 공사 등 대규모 및 준대규모점포 골목상권 ...,길고양이 TNR관련 안녕하세요.\n내년부터 서울시에서 길고양이 TNR을 실시한다는 ...


In [ ]:
# 토픽 모델링을 거친 '과 하는일' 데이터
df = pd.read_csv('..\\data\\0. 구_과_하는일_통합.csv')
df

,구,국,과,하는일,ex_token,ex_text,token,text,topic
0,광진구,행정국,총무과,광진구 의전지침 수립 및 현안 관리 등 신년인사회 구민의 날 정례조례 등 구 주요행...,"['광진구', '의전', '지침', '현안', '신년', '사회', '구민', '정...",광진구 의전 지침 현안 신년 사회 구민 정례 조례 행사 광진구 방위 협의회 일상 경...,"['광진구', '의전', '지침', '수립', '현안', '관리', '신년', '사...",광진구 의전 지침 수립 현안 관리 신년 사회 구민 정례 조례 주요 행사 추진 광진구...,32
1,광진구,행정국,자치행정과,동 행정 종합계획 동 직원 근무평정 조정 고향사랑기부제 동 지역책임제 사업 동 행정...,"['행정', '근무', '평정', '조정', '고향', '사랑', '기부', '책임...",행정 근무 평정 조정 고향 사랑 기부 책임 행정실 커뮤니티 공간 조성 적십자사 선거...,"['행정', '종합', '계획', '직원', '근무', '평정', '조정', '고향...",행정 종합 계획 직원 근무 평정 조정 고향 사랑 기부 지역 책임 사업 행정실 종합 ...,34
2,광진구,행정국,문화예술과,나루아트센터 리모델링 광진문화재단 지원 충무 보안 통합문화이용권 문화도시 사회복무요...,"['나루', '아트센터', '리모델링', '광진', '문화', '재단', '충무',...",나루 아트센터 리모델링 광진 문화 재단 충무 보안 문화 이용권 문화 도시 사회복무요...,"['나루', '아트센터', '리모델링', '광진', '문화', '재단', '지원',...",나루 아트센터 리모델링 광진 문화 재단 지원 충무 보안 통합 문화 이용권 문화 도시...,33
3,광진구,행정국,체육진흥과,체육인 통합 커뮤니티센터 및 구민 체육 증진센터 조성 장애인체육회 설립 자양체육관 ...,"['체육인', '커뮤니티', '구민', '체육', '증진', '조성', '장애인',...",체육인 커뮤니티 구민 체육 증진 조성 장애인 체육회 설립 자양 체육관 스포츠 클럽 ...,"['체육인', '통합', '커뮤니티', '센터', '구민', '체육', '증진', ...",체육인 통합 커뮤니티 센터 구민 체육 증진 센터 조성 장애인 체육회 설립 자양 체육...,12
4,광진구,행정국,교육지원과,구립도서관 국시비보조사업 다문화서비스 서울북스타트 도서관과 함께 책읽기 독서동아리 ...,"['구립', '도서관', '시비', '보조', '다문화', '서비스', '스타트',...",구립 도서관 시비 보조 다문화 서비스 스타트 도서관 독서 동아리 구립 도서관 일자리...,"['구립', '도서관', '시비', '보조', '사업', '다문화', '서비스', ...",구립 도서관 시비 보조 사업 다문화 서비스 서울 스타트 도서관 독서 동아리 운영 지...,25
...,...,...,...,...,...,...,...,...,...
929,노원구,보건소,생활보건과,방역소독사업 관련 업무 소독의무시설 소독업소 포함 관리 직원 복무 및 수당 에이즈 ...,"['방역', '소독', '소독', '의무', '소독', '업소', '포함', '복무...",방역 소독 소독 의무 소독 업소 포함 복무 수당 에이즈 매개 감염병 선별 진료소 선...,"['방역', '소독', '사업', '관련', '업무', '소독', '의무', '시설...",방역 소독 사업 관련 업무 소독 의무 시설 소독 업소 포함 관리 직원 복무 수당 에...,3
930,노원구,보건소,의약과,대직자 강미숙 재난안전관리에 관한 사항 구급차 응급의료기관 관리 장기기증 헌혈사랑 ...,"['대직자', '미숙', '재난', '안전', '구급차', '응급', '의료', '...",대직자 미숙 재난 안전 구급차 응급 의료 장기 기증 헌혈 사랑 안경 치과 공소 의료...,"['대직자', '미숙', '재난', '안전', '관리', '사항', '구급차', '...",대직자 미숙 재난 안전 관리 사항 구급차 응급 의료 기관 관리 장기 기증 헌혈 사랑...,3
931,노원구,보건소,보건지소,보건지소 총괄 업무 월계 공릉지소 예산 회계 직원 수당관리 구강보건사업 전반 한의약...,"['보건', '지소', '공릉', '지소', '수당', '구강', '보건', '전반...",보건 지소 공릉 지소 수당 구강 보건 전반 의약 건강 증진 한방 진료 상담 의약 건...,"['보건', '지소', '총괄', '업무', '공릉', '지소', '예산', '회계...",보건 지소 총괄 업무 공릉 지소 예산 회계 직원 수당 관리 구강 보건 사업 전반 의...,3
932,도봉구,미래환경국,자원순환과,자원순환과 업무 총괄자원순환팀 업무 총괄과서무 예산 회계청소차량 정비 및 정비시설 ...,"['자원', '순환', '자원', '순환', '서무', '청소', '정비', '정비...",자원 순환 자원 순환 서무 청소 정비 정비 봉투 판매소 지정 취소 공중화장실 종량제...,"['자원', '순환', '업무', '총괄', '자원', '순환', '업무', '총괄...",자원 순환 업무 총괄 자원 순환 업무 총괄 서무 예산 회계 청소 차량 정비 정비 시...,11


In [ ]:
topic_df = df[['구', '과', 'topic']]
topic_df.columns = ['gu_name', 'resp_dept', 'topic']
topic_df

,gu_name,resp_dept,topic
0,광진구,총무과,32
1,광진구,자치행정과,34
2,광진구,문화예술과,33
3,광진구,체육진흥과,12
4,광진구,교육지원과,25
...,...,...,...
929,노원구,생활보건과,3
930,노원구,의약과,3
931,노원구,보건지소,3
932,도봉구,자원순환과,11


In [ ]:
merge_df = pd.merge(minwon_df, topic_df, on=['gu_name', 'resp_dept'])
merge_df

,req_id,req_title,req_p,req_date,req_content,resp_dept,resp_date,resp_writer,resp_content,page_num,gu_name,국,하는일,req_title_content,topic
0,445530.0,양재천 너구리(?) 출현,홍영규,2023-07-31 08:42:53,어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 하다가 숲속에 너구리 여러...,공원녹지과,2023-08-01 18:01:12,이지영,"안녕하십니까? 우리 구정에 많은 관심과 애정을 가져주심에 감사드리며, 양재천에 서식...",1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...,16
1,440728.0,갈매기공원 도랑 수리 등,양세성,2023-07-15 11:22:49,1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이 여기저기 불거져 물이 고...,공원녹지과,2023-07-25 13:10:01,정우영,1. 우리 구정발전을 위하여 많은 관심과 애정을 가져주신데 감사드립니다.\n\n2....,1,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,갈매기공원 도랑 수리 등 1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이...,16
2,437057.0,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요",백승도,2023-07-04 14:37:02,1. 세곡교에서 강남신동아파밀리에1단지로 들어오는 보행로상에 가로등이 설치되어 있으...,공원녹지과,2023-07-07 11:23:24,이지영,"1. 우리 구정에 깊은 관심을 가져주심에 감사드리며, 세곡1교에서 신동아파밀리에 1...",2,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요 1. 세곡교에서 강남신동아파밀리에...",16
3,436489.0,급식소 철거 계고장을 철회해 주세요,정미혜,2023-07-02 22:09:50,서울시 강남구 청담동 나들목 급식소 철거 계고장에 대한 원만한 해결을 부탁드립니다....,공원녹지과,2023-07-11 08:42:18,이지현,"1. 우리 구정 발전을 위하여 많은 관심과 애정을 가져주신데 감사드리며, 청담나들목...",2,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,급식소 철거 계고장을 철회해 주세요 서울시 강남구 청담동 나들목 급식소 철거 계고장...,16
4,431715.0,고압선 접촉 가로수 가지치기,권오민,2023-06-15 09:25:42,"가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, 단선의 위험이 있으니 빨리 가...",공원녹지과,2023-06-22 09:44:17,조성현,1. 항상 우리 구정 발전에 많은 관심을 가져주심에 감사드립니다.\n2. 귀하께서 ...,3,강남구,도시환경국,근린 주제공원 조성 변경 에 관한 도시계획 업무 총괄 도시관리 변경 계획 공원조성 ...,"고압선 접촉 가로수 가지치기 가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, ...",16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88034,61556.0,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체),박인태,2020-03-03 13:37:23,1. 본 민원인은 살균 소독제를 제조하고 있습니다.\n\n2. 살균 소독제의 제품명...,위생과,2020-03-06 17:32:52,공용기,"안녕하십니까?\n코로나19 감염 확산을 막기 위해 우리구는 선별진료소, 터미널, 지...",63,중랑구,보건소,식품진흥기금 음식문화개선사업 행정심판 및 행정소송 음식점 원산지 관리 음식점 원산지...,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체) 1...,10
88035,48831.0,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다.,정○○,2019-07-03 13:49:33,봉우재로 33길에있는 음식점들 노상(서울시 주차장자리)에 가판피고 음식팔고 술팔고 ...,위생과,2019-07-11 10:25:45,고영배,안녕하십니까?\n평소 우리 구정에 많은 관심과 애정을 가져주셔서 감사합니다.\n중랑...,72,중랑구,보건소,식품진흥기금 음식문화개선사업 행정심판 및 행정소송 음식점 원산지 관리 음식점 원산지...,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다. 봉우재로 33길에있...,10
88036,48540.0,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요,김○○,2019-06-26 17:17:43,"본인은 일시적인 내용을 보고, 야기 하는 것이 아닌 중화동에서 초,중,고를 나온만큼...",위생과,2019-07-03 13:19:35,고영배,안녕하십니까?\n평소 우리 구정에 많은 관심과 애정을 가져주셔서 감사합니다.\n현재...,73,중랑구,보건소,식품진흥기금 음식문화개선사업 행정심판 및 행정소송 음식점 원산지 관리 음식점 원산지...,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요 본...,10
88037,80967.0,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다.,윤새나,2021-01-04 09:50:00,"오늘난 기사입니다.. 전국 안전지수 하위 지역에 서울 중 가장 낮은 지역, 전국으로...",기획예산과,2021-01-28 13:49:27,박은정,○ 안녕하십니까? 구정에 많은 관심과 애정을 가지고 소중한 의견을 주신 귀하께 감사...,43,중랑구,기획재정국,지속가능발전 기본전략 및 추진계획 국서무 당정협의회 구정질의 핵심지표 개발 및 평가...,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다. 오늘난 기사입니다.....,34


In [ ]:
# 필요한 컬럼만 가지고 학습 데이터 만들기
train_col = ['req_title_content', 'resp_dept', 'gu_name', 'topic']
train_data = merge_df[train_col]
train_data

,req_title_content,resp_dept,gu_name,topic
0,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...,공원녹지과,강남구,16
1,갈매기공원 도랑 수리 등 1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이...,공원녹지과,강남구,16
2,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요 1. 세곡교에서 강남신동아파밀리에...",공원녹지과,강남구,16
3,급식소 철거 계고장을 철회해 주세요 서울시 강남구 청담동 나들목 급식소 철거 계고장...,공원녹지과,강남구,16
4,"고압선 접촉 가로수 가지치기 가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, ...",공원녹지과,강남구,16
...,...,...,...,...
88034,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체) 1...,위생과,중랑구,10
88035,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다. 봉우재로 33길에있...,위생과,중랑구,10
88036,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요 본...,위생과,중랑구,10
88037,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다. 오늘난 기사입니다.....,기획예산과,중랑구,34


In [ ]:
# 컬럼 이름 변경
train_data.columns = ['req_content', 'resp_dept', 'gu_name', 'topic']
train_data

,req_content,resp_dept,gu_name,topic
0,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...,공원녹지과,강남구,16
1,갈매기공원 도랑 수리 등 1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이...,공원녹지과,강남구,16
2,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요 1. 세곡교에서 강남신동아파밀리에...",공원녹지과,강남구,16
3,급식소 철거 계고장을 철회해 주세요 서울시 강남구 청담동 나들목 급식소 철거 계고장...,공원녹지과,강남구,16
4,"고압선 접촉 가로수 가지치기 가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, ...",공원녹지과,강남구,16
...,...,...,...,...
88034,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체) 1...,위생과,중랑구,10
88035,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다. 봉우재로 33길에있...,위생과,중랑구,10
88036,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요 본...,위생과,중랑구,10
88037,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다. 오늘난 기사입니다.....,기획예산과,중랑구,34


In [ ]:
train_data.to_csv('../data/train_data.csv', index=False, encoding='utf-8-sig')

In [ ]:
temp_df = pd.read_csv('../data/train_data.csv')
temp_df

,req_content,resp_dept,gu_name,topic
0,양재천 너구리(?) 출현 어제 양재천 오후 9:30분 쯤 계양 중학교 후면 산책을 ...,공원녹지과,강남구,16
1,갈매기공원 도랑 수리 등 1. 8차 현대아파트 옆 공원 물빠짐 도랑 수리\n도수관이...,공원녹지과,강남구,16
2,"가로등 불빛을 가리는 나무를 전지, 전정해 주세요 1. 세곡교에서 강남신동아파밀리에...",공원녹지과,강남구,16
3,급식소 철거 계고장을 철회해 주세요 서울시 강남구 청담동 나들목 급식소 철거 계고장...,공원녹지과,강남구,16
4,"고압선 접촉 가로수 가지치기 가로수가 고압선 전선에 접촉해 있습니다.\n\n정전, ...",공원녹지과,강남구,16
...,...,...,...,...
88034,코로나19에 대응할 살균소독제를 무상으로 기부하겠습니다. (살균소독제 제조업체) 1...,위생과,중랑구,10
88035,상봉동 봉우재로 33길 음식점에서 길에다 노상피고 못살겠습니다. 봉우재로 33길에있...,위생과,중랑구,10
88036,중랑역 포장마차 위생상태 현황 점검 여부 및 카드결제 거부로 인한 현금결제 강요 본...,위생과,중랑구,10
88037,중랑구 치안을 위한 중랑구청 차원의 근본적인 정책시행바랍니다. 오늘난 기사입니다.....,기획예산과,중랑구,34
